In [7]:
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf

import tensorflow as tf

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score

Dense = tf.keras.layers.Dense
Input = tf.keras.layers.Input
Sequential = tf.keras.Sequential
MeanSquaredError = tf.keras.losses.MeanSquaredError
BinaryCrossentropy = tf.keras.losses.BinaryCrossentropy
Sigmoid = tf.keras.activations.sigmoid

file_path = os.getcwd()

conf = OmegaConf.load(os.path.join(file_path, "..", "src", "config.yml"))

data_path = os.path.join(file_path, "..", "data", "UCI_Credit_Card.csv")

df = pd.read_csv(data_path)

lambdas = [0, 1e-5, 1e-4, 5e-4, 1e-3]

In [8]:
def create_percent_paid_features(df):
    for i in range(1, 7):
        df[f"PCT_PAID_{i}"] = np.where(
            df[f"BILL_AMT{i}"] > 0,
            df[f"PAY_AMT{i}"] / df[f"BILL_AMT{i}"],
            0
        )

    df["PCT_PAID_MEAN"] = df[[f"PCT_PAID_{i}" for i in range(1, 7)]].mean(axis=1)
    return df


def create_credit_utilization(df):
    bill_cols = [f"BILL_AMT{i}" for i in range(1, 7)]
    df["BILL_MEAN"] = df[bill_cols].mean(axis=1)

    df["CREDIT_UTILIZATION"] = np.where(
        df["LIMIT_BAL"] > 0,
        df["BILL_MEAN"] / df["LIMIT_BAL"],
        0
    )
    return df


def create_pay_delay_features(df):
    pay_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
    df["PAY_DELAY_MEAN"] = df[pay_cols].mean(axis=1)
    df["PAY_DELAY_MAX"] = df[pay_cols].max(axis=1)
    return df


def feature_engineering(df):
    df = df.copy()
    df = create_percent_paid_features(df)
    df = create_credit_utilization(df)
    df = create_pay_delay_features(df)
    return df


In [9]:
def prepare_data(df, params):

    df = feature_engineering(df)
    # ===============================
    # TARGET
    # ===============================
    y = df['default.payment.next.month']


    # ===============================
    # FEATURES
    # ===============================
    X = df[
        [
            'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
            'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
            'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
            'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
            'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
            'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
            'PCT_PAID_MEAN',
            'CREDIT_UTILIZATION',
            'PAY_DELAY_MEAN',
            'PAY_DELAY_MAX'
        ]
    ]
    
    # ===============================
    # DEFINIÇÃO DOS TIPOS
    # ===============================
    binary_features = ['SEX']
    categorical_features = ['EDUCATION', 'MARRIAGE']
    numerical_features = [col for col in X.columns if col not in binary_features + categorical_features]
    # numerical_features = [
    #     'LIMIT_BAL',
    #     'AGE',
    #     'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    #     'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',
    #     'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
    #     'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
    #     'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
    # ]
    
    # ===============================
    # PREPROCESSAMENTO
    # ===============================
    preprocessor = ColumnTransformer(
        transformers=[
            ('bin', MinMaxScaler(), binary_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
            ('num', MinMaxScaler(), numerical_features)
        ]
    )
    
    # ===============================
    # SPLIT
    # ===============================
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=params["test_size"],
        random_state=params["random_state"],
        stratify=y
    )
    
    # ===============================
    # TRANSFORMAR DADOS
    # ===============================
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return X_train, X_test, y_train, y_test

In [10]:
def cross_validate_lambda(X, y, lambdas, params):

    skf = StratifiedKFold(
        n_splits=params["cv_folds"],
        shuffle=True,
        random_state=params["random_state"]
    )

    best_lambda = None
    best_auc = -np.inf

    for lambda_ in lambdas:
        aucs = []

        for train_idx, val_idx in skf.split(X, y):

            X_tr, X_val = X[train_idx], X[val_idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            model = Sequential([
                Dense(64, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
                Dense(32, activation='relu',
                      kernel_regularizer=tf.keras.regularizers.l2(lambda_)),
                Dense(1, activation='linear')
            ])

            model.compile(
                optimizer=tf.keras.optimizers.Adam(
                    learning_rate=params["learning_rate"]
                ),
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
            )

            model.fit(
                X_tr,
                y_tr,
                epochs=params["epochs_cross"],
                verbose=params["verbose"]
            )

            logits = model(X_val)
            y_pred_proba = tf.nn.sigmoid(logits).numpy().ravel()

            auc = roc_auc_score(y_val, y_pred_proba)
            aucs.append(auc)

        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)

        print(f"[CV] λ={lambda_:.5f} | AUC={mean_auc:.4f} ± {std_auc:.4f}")

        if mean_auc > best_auc:
            best_auc = mean_auc
            best_lambda = lambda_


    return best_lambda

In [11]:
def train_final_model(X_train, X_test, y_train, y_test, best_lambda, params):

    model = Sequential([
        Dense(64, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
        Dense(32, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(best_lambda)),
        Dense(1, activation='linear')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=params["learning_rate"]
        ),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    )

    model.fit(
        X_train,
        y_train,
        epochs=params["epochs"],
        verbose=params["verbose"]
    )

    logits = model(X_test)
    y_pred_proba = tf.nn.sigmoid(logits).numpy().ravel()
    y_pred = (y_pred_proba >= 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    print("\n📊 TEST SET FINAL")
    print(f"Accuracy: {acc:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")

    return model


In [12]:
def main():
     
    # Preparação dos dados
    X_train, X_test, y_train, y_test = prepare_data(df, conf["parameters"])

    best_lambda = cross_validate_lambda(X_train, y_train, lambdas, conf["parameters"])

    print(f"🏆 Melhor lambda escolhido via CV: {best_lambda:.1e}")

    train_final_model(X_train, X_test, y_train, y_test, best_lambda, conf["parameters"])

if __name__ == "__main__":
    main()


[CV] λ=0.00000 | AUC=0.7685 ± 0.0062
[CV] λ=0.00001 | AUC=0.7673 ± 0.0070
[CV] λ=0.00010 | AUC=0.7637 ± 0.0071
[CV] λ=0.00050 | AUC=0.7515 ± 0.0069
[CV] λ=0.00100 | AUC=0.7495 ± 0.0140
🏆 Melhor lambda escolhido via CV: 0.0e+00

📊 TEST SET FINAL
Accuracy: 0.8130
ROC-AUC: 0.7673
